In [26]:
import drawSvg as draw
import numpy as np
import pandas as pd
import os
from hilbert import decode, encode

In [18]:
def get_player_data(player_df, feats):
    player_game_ids = list(player_df['game_id'])
    
    player_data = []
    
    for game_id in range(1,83):
        if game_id not in player_game_ids:
            player_data.append([0]*len(feats))
        else:
            player_game_id_data = [player_df[player_df.game_id == game_id][feat].values[0] for feat in feats]
            #player_game_id_data = []
            #for feat in feats:
            #    data = player_df[player_df.game_id == game_id][feat].values[0]
            #    player_game_id_data.append(data)
            player_data.append(player_game_id_data)
        
    return np.array(player_data)
    
    
def build_3d_player_matrix(data_path, feats, team=None, min_gp=0):
    player_matrix = []
    player_names  = []
    
    for fname in os.listdir(data_path):
        player_df = pd.read_csv(data_path+'/'+fname, sep='|')
        
        if player_df.shape[0] < min_gp:
            continue

        if team:
            if team not in player_df.team.values: continue
        
        player_names.append(fname)
        player_data = get_player_data(player_df, feats)
        
        player_matrix.append(player_data)
    
    player_matrix = np.array(player_matrix)   
    
    return player_matrix,player_names


In [19]:
def zcurve(x,y):
    xx = x
    yy = y
    level = 0
    result = 0
    while xx > 0 or yy > 0:
        x_bit = 1 if (xx & 1) else 0
        y_bit = 1 if (yy & 1) else 0
        result += (x_bit + (y_bit << 1)) << (2*level)
        xx = xx >> 1
        yy = yy >> 1
        level += 1
    return result

def zcurve_inv(i):
    ii = i
    x = 0
    y = 0
    level = 0
    while ii > 0:
        x_bit = 1 if (ii & 1) else 0
        y_bit = 1 if (ii & 2) else 0
        x += x_bit << level
        y += y_bit << level
        level += 1
        ii = ii >> 2
    return (x,y)

In [20]:
def map_points(max_p, p, cell_width, cell_height, cell_center):
    x_p = 0
    y_p = (p*(cell_height/2))/max_p
    
    return cell_center[0]+x_p, cell_center[1]+y_p

def map_assists(max_a, a, cell_width, cell_height, cell_center):
    x_a = (a*(-cell_width/2))/max_a
    y_a = (a*(-cell_height/2))/max_a
    
    return cell_center[0]+x_a, cell_center[1]+y_a

def map_rebounds(max_r, r, cell_width, cell_height, cell_center):
    x_r = (r*(cell_width/2))/max_r
    y_r = (r*(-cell_height/2))/max_r
    
    return cell_center[0]+x_r, cell_center[1]+y_r

In [115]:
def draw_glyph_matrix(data, width, height, cell_width, cell_height):
    max_p = np.max(data[:,:,0])
    max_a = np.max(data[:,:,1])
    max_r = np.max(data[:,:,2])

    canvas = draw.Drawing(width, height, origin=(0,0), displayInline=False)

    for i in range(data.shape[0]):
        row_upper_points = []
        for j in range(data.shape[1]):
            cell_center_x = (j*cell_width)+(cell_width/2)
            cell_center_y = (i*cell_height)+(cell_height/2)
            cell_center = [cell_center_x, cell_center_y]
            
            radius = 1
            center_dot = draw.Circle(cell_center_x, cell_center_y, radius,
                                     fill='black', stroke_width=2, stroke='black')
            
            
            end = data.shape[0]-1
            p, a, r = data[end-i][j][0], data[end-i][j][1], data[end-i][j][2]

            x_p, y_p = map_points(max_p, p, cell_width, cell_height, cell_center)
            row_upper_points.append(x_p)
            row_upper_points.append(y_p)
            x_a, y_a = map_assists(max_a, a, cell_width, cell_height, cell_center)
            x_r, y_r = map_rebounds(max_r, r, cell_width, cell_height, cell_center)


            poly = draw.Lines(x_p, y_p,
                              x_a, y_a,
                              x_r, y_r,
                              close=True, fill="#0000FF55")

#             text_p = draw.Text(str(p), 10, cell_center[0]-5, y_p+5)
#             text_a = draw.Text(str(a), 10, x_a, cell_center[1]-10)
#             text_r = draw.Text(str(r), 10, x_r, cell_center[1]-10)

            canvas.append(center_dot)
            canvas.append(poly)
#             canvas.append(text_p)
#             canvas.append(text_a)
#             canvas.append(text_r)
        
        p = draw.Path(stroke_width=1, stroke='red', fill_opacity=0)

        for i in range(0, len(row_upper_points), 2):
            if i==0:
                p.M(row_upper_points[i], row_upper_points[i+1])
            else:
                p.L(row_upper_points[i], row_upper_points[i+1])
        canvas.append(p)

    return canvas

In [116]:
lbj, lbj_names = build_3d_player_matrix(r'../../data/NBA/test_matrix', ['points','assists','rebounds'])
lbj_10 = lbj[:,:,:]

#width  = 2000
#height = 200

cell_width  = 40
cell_height = 40

width = cell_width * lbj_10.shape[1]
height = cell_height * lbj_10.shape[0]
print(width, height)

lbj10_glyph_matrix = draw_glyph_matrix(lbj_10, width, height, cell_width, cell_height)
lbj10_glyph_matrix

3280 160


In [117]:
feats = ['points','assists','rebounds']
gsw_data, gsw_names = build_3d_player_matrix(r"../../data/NBA/nba_1819/nba_players_1819", feats,
                                             team='Team.GOLDEN_STATE_WARRIORS')
gsw_data.shape

(17, 82, 3)

In [118]:
cell_width  = 40
cell_height = 40
gsw_data = gsw_data[:,:20,:]
width = cell_width * gsw_data.shape[1]
height = cell_height * gsw_data.shape[0]
print(width, height)
print(gsw_names)
gsw_glyph_matrix = draw_glyph_matrix(gsw_data, width, height, cell_width, cell_height)
gsw_glyph_matrix

800 680
['Alfonzo_McKinnie', 'Andrew_Bogut', 'Andre_Iguodala', 'Damian_Jones', 'Damion_Lee', 'DeMarcus_Cousins', 'Draymond_Green', 'Jacob_Evans', 'Jonas_Jerebko', 'Jordan_Bell', 'Kevin_Durant', 'Kevon_Looney', 'Klay_Thompson', 'Marcus_Derrickson', 'Quinn_Cook', 'Shaun_Livingston', 'Stephen_Curry']


# Tests

In [67]:
c = draw.Drawing(100, 100, origin=(0,0), displayInline=False)

p = draw.Path(stroke_width=2, stroke='red', fill_opacity=0)
top_pts = [30,20,60,15,90,30]

for i in range(0, len(top_pts), 2):
    if i==0:
        p.M(top_pts[i], top_pts[i+1])
    else:
        p.L(top_pts[i], top_pts[i+1])
c.append(p)
c

In [164]:
c = draw.Drawing(200, 200, origin=(0,0), displayInline=False)

cx = 100
cy = 100
r  = 100
deg_start = 0
deg_end = 90

# p = draw.Path(stroke='black', stroke_width=1)
# p.M(cx,cy)
# p.L(cx,cy+r)
# p.arc(cx,cy,r,deg_start,deg_end,cw=False)
# p.Z()

# Draw another shape to fill with the same gradient
p = draw.Path(fill='green', stroke='red', stroke_width=1)
# p.arc(cx,cy,r,deg_end,deg_start)
p.arc(cx,cy,r,deg_start,deg_end,cw=True, includeM=True)
p.Z()

c.append(p)

c

In [82]:
cell_width = 200
cell_height = 200
cell_center = [100,100]

c = draw.Drawing(cell_width, cell_height, origin=(0,0), displayInline=False)

max_p = 40
max_a = 15
max_r = 20

p, a, r = 25, 10, 5

x_p, y_p = map_points(max_p, p, cell_width, cell_height, cell_center)
x_a, y_a = map_assists(max_a, a, cell_width, cell_height, cell_center)
x_r, y_r = map_rebounds(max_r, r, cell_width, cell_height, cell_center)

fl = draw.Lines(cell_center[0],cell_center[1],
                x_p, y_p,
                close=False,
                stroke="red")
sl = draw.Lines(cell_center[0],cell_center[1],
                x_a, y_a,
                close=False,
                stroke="blue")
tl = draw.Lines(cell_center[0],cell_center[1],
                x_r, y_r,
                close=False,
                stroke="green")               
                
c.append(fl)
c.append(sl)
c.append(tl)

c

In [203]:
cell_Width, cell_height = 200, 200
cell_center = [100,100]

d = draw.Drawing(cell_width, cell_height, origin=(0,0), displayInline=False)

cx = 100
cy = 100

max_p = 40
max_a = 15
max_r = 20

p, a, r = 25, 10, 5

x_p, y_p = map_points(max_p, p, cell_width, cell_height, cell_center)
x_a, y_a = map_assists(max_a, a, cell_width, cell_height, cell_center)
x_r, y_r = map_rebounds(max_r, r, cell_width, cell_height, cell_center)

p = draw.Path(fill='red')
p.arc(cx, cy, y_p, 75, 105)
p.arc(cx, cy, 0, 105, 75, cw=True, includeL=True)
p.Z()

a = draw.Path(fill='blue')
a.arc(cx, cy, y_a, 195, 225)
a.arc(cx, cy, 0, 225, 195, cw=True, includeL=True)
a.Z()

r = draw.Path(fill='green')
r.arc(cx, cy, y_r, 315, 345)
r.arc(cx, cy, 0, 345, 315, cw=True, includeL=True)
r.Z()

d.append(p)
d.append(a)
d.append(r)

# Display
d